# Problem 3: Solving discrete logarithm problem using index calculus method

In [92]:
import math
from typing import List
import random
import fieldmath
import numpy as np

In [93]:
p = 10930889
alpha = 2317547
alpha_order = 59407
beta = 5467427

## Using exhaustive search

In [94]:
alpha_i = 1
for i in range(1,alpha_order):
    alpha_i = alpha_i*alpha % p
    if alpha_i == beta:
        print(alpha,"**", i, "=", alpha_i, "beta =", beta)
        break
    

2317547 ** 2047 = 5467427 beta = 5467427


Now that we have made sure that beta = 5467427 is in the subgroup generated by alpha = 2317547, we can use index calculus to find the discrete log of beta.

## Preprocessing step

In [95]:
factor_base = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31]
B = len(factor_base)

### Computing the discrete log of the elements in the factor base

In [96]:
def factor(n: int, factor_base: List[int]) -> List[int]:
    if not n or not factor_base:
        return []
    
    exponents = list()
    
    for fact in factor_base:
        if fact == -1:
            if abs(n) == n:
                exponents.append(0)
            else:
                n = abs(n)
                exponents.append(1)
        else:
            i = 0
            while math.gcd(fact, n) != 1:
                n = n//fact
                i += 1

            exponents.append(i)
    
    if n != 1:
        return []
    else:
        return exponents


In [97]:
# testing factor fct:
print(factor(243, [2, 3]))
print(factor(243, [2]))
print(factor(-243, [-1, 2, 3]))

[0, 5]
[]
[1, 0, 5]


In [98]:
def power(x, y, m) : 
      
    if (y == 0) : 
        return 1
          
    p = power(x, y // 2, m) % m 
    p = (p * p) % m 
  
    if(y % 2 == 0) : 
        return p  
    else :  
        return ((x * p) % m) 

    

### Randomly finding C = B +10 numbers that factor over the factor base

In [127]:
C = B

# This matrix is constructed in such that
# it contains C lines and B + 1 columns
# and the first B columns are the powers of
# the decomposition in the factor_base
# and the last column is the power of alpha
# we are considering

alpha_powers = list()
matrix_to_solve = list()
while(len(matrix_to_solve) < C):
    a = random.randint(0, alpha_order)
    if a in alpha_powers:
        continue
    
    alpha_a = power(alpha, a, p)
    factorization = factor(alpha_a, factor_base)
    if factorization:
        factorization.append(a)
        matrix_to_solve.append(factorization)
        alpha_powers.append(a)
matrix_to_solve = np.array(matrix_to_solve)



In [128]:
alpha_powers

[24685, 32882, 36216, 45530, 57740, 50953, 49579, 35277, 20318, 1626, 55718]

In [130]:
# using the fieldmath library to reduce to row echelon form
M, N = len(matrix_to_solve), len(matrix_to_solve[0])
f = fieldmath.PrimeField(alpha_order)
mat = fieldmath.Matrix(M, N, f)
for i in range(M):
    for j in range(N):
        mat.set(i, j, matrix_to_solve[i][j])

mat.reduced_row_echelon_form()
reduced_matrix = list(np.zeros((M, N), dtype = int))

In [131]:
reduced_matrix = np.zeros((len(matrix_to_solve), 
                           len(matrix_to_solve[0])),
                          dtype = int)
for i in range(mat.row_count()):
    for j in range(mat.column_count()):
        reduced_matrix[i, j] = mat.get(i, j)
    
reduced_matrix

array([[    1,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0, 26152],
       [    0,     1,     0,     0,     0,     0,     0,     0,     0,
            0,     0,  6627],
       [    0,     0,     1,     0,     0,     0,     0,     0,     0,
            0,     0, 26841],
       [    0,     0,     0,     1,     0,     0,     0,     0,     0,
            0,     0,  8299],
       [    0,     0,     0,     0,     1,     0,     0,     0,     0,
            0,     0, 31335],
       [    0,     0,     0,     0,     0,     1,     0,     0,     0,
            0,     0, 13320],
       [    0,     0,     0,     0,     0,     0,     1,     0,     0,
            0,     0, 54311],
       [    0,     0,     0,     0,     0,     0,     0,     1,     0,
            0,     0, 18780],
       [    0,     0,     0,     0,     0,     0,     0,     0,     1,
            0,     0,  5616],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            1,     0

Now that we found the logarithms for all the elements in the factor base, we store those elements

We re-run the above cells until the cell below returns True, that means that we managed to find a unique solution for the congruences

In [137]:
# check if the reduced matrix has the form of a linear solution:
M = reduced_matrix[:, :-1]
(M.shape[0] == M.shape[1]) and (M == np.eye(M.shape[0])).all()

True

In [132]:
discrete_logs = dict(zip(factor_base, reduced_matrix[:, -1]))
discrete_logs

{2: 26152,
 3: 6627,
 5: 26841,
 7: 8299,
 11: 31335,
 13: 13320,
 17: 54311,
 19: 18780,
 23: 5616,
 29: 41444,
 31: 8276}

## Second Step: computing the discrete logarithm of beta

In [145]:
factored = False
log_beta = None
while not factored:
    s = random.randint(1, p-2)
    alpha_pow_s = power(alpha, s, p)
    gamma = (beta * alpha_pow_s) % p
    gamma_factors = factor(gamma, factor_base)
    if gamma_factors:
        log_beta = 0
        for i in range(len(gamma_factors)):
            log_beta += gamma_factors[i]*discrete_logs[factor_base[i]]
        log_beta = (log_beta - s) % alpha_order
        factored = True
        break


log_beta            

2047